In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import lru_cache

In [3]:
@lru_cache(maxsize=None)  
def fetch_data_from_source(parquet_file):
    df = pd.read_parquet(f"https://data3001-racing.s3.ap-southeast-2.amazonaws.com/{parquet_file}")
    return df

def fetch_data_csv(parquet_file):
    df = pd.read_csv(f"https://data3001-racing.s3.ap-southeast-2.amazonaws.com/{parquet_file}")
    return df

In [9]:
# Inputs: N/A
# Outputs: df with both datasets appended to each other (sorted by "SESSION_IDENTIFIER" then "FRAME")
# NOTE: 'SECTOR' has been changed from {0,1,2} to {1,2,3}

def fetch_and_merge():
    df22 = fetch_data_from_source('f1sim-data-2022.parquet')
    df23 = fetch_data_from_source('f1sim-data-2023.parquet')
    df = pd.concat([df22, df23], ignore_index=True)
    df.dropna(inplace=True)
    df.sort_values(by=["SESSION_IDENTIFIER", "FRAME"], inplace=True)
    df.reset_index(drop=True, inplace=True)
    df["SECTOR"] = df["SECTOR"] + 1
    return df

In [5]:
# Inputs: Sector {1,2,3}
# Outputs: df of records with data from that sector

def split_sector(s):
    df = fetch_and_merge()
    df = df[df["SECTOR"]==s]
    return df

In [6]:
# Inputs: df, time
# Outputs: df with records where lap time is under time given
def lap_under(df, t):
    df = df[df["LAP_TIME_MS"]<t]
    return df

In [ ]:
# Inputs: sector {1,2,3}, time (in ms)
# Outputs: df where the time set in the sector is less than the time given

def sector_under(s, t):
    df = split_sector(s)
    if s==1:
        str = 'SECTOR_1_TIME_MS'
    elif s==2:
        str = 'SECTOR_2_TIME_MS'
    elif s==3:
        str = 'SECTOR_3_MS'

    df = df[df[str]<t]
    return df

In [ ]:
# Inputs: turn
# Output: df where all data is in the turn given

def split_turns(t):
    df = fetch_and_merge()
    t_df = fetch_data_csv('f1sim-ref-turns.csv')
    t = t-1
    x1 = t_df["CORNER_X1"][t]
    x2 = t_df["CORNER_X2"][t]
    y1 = t_df["CORNER_Y1"][t]
    y2 = t_df["CORNER_Y2"][t]
    df = df[(df["WORLDPOSX"] > x1) & (df["WORLDPOSX"] < x2)]
    df = df[(df["WORLDPOSY"] > y1) & (df["WORLDPOSY"] < y2)]
    return df